# Ejercicio en clase: ETL del clima de Seattle + Postgres (Pandas)

## Objetivo
Construir un pequeño programa: **ingesta → limpieza → enriquecimiento → agregaciones → persistencia en Postgres**.

- Usar **Pandas** para todas las transformaciones.
- Usar **SQLAlchemy** o **Psycopg2** para la persistencia
- Usar **Postgres** en una instancia Docker local.

**NOTA**: Todas las cosas que aquí se os piden son **mínimas**. Podéis (y debéis) ir más allá si queréis. Además, podeis usar vustro criterio no solo para definir como implementar cada paso, sino para proponer mejoras o pasos adicionales o directamente cambiarlos o no hacerlos. Todo esto, eso sí, debe quedar documentado en el README y debidamente justificado.

## Dataset
- URL: `https://cdn.jsdelivr.net/npm/vega-datasets@3.2.1/data/seattle-weather.csv`
- Columnas esperadas: `date, precipitation, temp_max, temp_min, wind, weather`.
- Unidades métricas: temperatura (°C), precipitación (mm), viento (m/s).
- Periodo aproximado: 2012–2015.

## Entregables
1) Un notebook con todos los pasos (en **Pandas**).
2) Tablas en Postgres (en vuestra instancia Docker local):
   - `seattle_weather_raw`
   - `seattle_weather_clean`
   - Agregadas: `sw_agg_by_year`, `sw_agg_by_month`, `sw_agg_by_weather`
3) Un README breve explicando supuestos y decisiones.


## Tareas

### 1) Lectura del dataset (ingesta)
- Lee el CSV **directamente desde la URL**.
- Asegura tipos correctos:
  - `date` → fecha
  - `precipitation`, `temp_max`, `temp_min`, `wind` → numéricos
  - `weather` → string en minúsculas
- Guarda **sin tocar** en Postgres como `seattle_weather_raw`.

### 2) Limpieza (tareas concretas)
Aplica y documenta estas reglas:
- **Duplicados**: elimina duplicados por `date`.
- **Valores inválidos**:
  - `precipitation < 0` → fija a 0.
  - `wind < 0` → fija a `NULL` o 0 (justifica).
  - Si `temp_min > temp_max` → **intercámbialas** (swap).
- **Normalización categórica**:
  - `weather`: trim + minúsculas; normaliza variantes a un conjunto acotado (p.ej., `['sun', 'rain', 'drizzle', 'snow', 'fog']`).

Guarda el resultado como `seattle_weather_clean`.

### 3) Nuevas variables (enriquecimiento)
Crea, al menos, estas columnas:
- `year` (año), `month` (1–12), `dow` (día de la semana en texto).
- `season` (invierno: 12–2, primavera: 3–5, verano: 6–8, otoño: 9–11).
- `temp_range = temp_max - temp_min`.
- `temp_mean = (temp_max + temp_min) / 2`.
- `is_wet_day = precipitation > 0`.

### 4) Tablas agrupadas (3 tablas)
Genera **tres** dataframes agregados con **groupby + agregados simples** y persístelos:

1. `sw_agg_by_year` (por `year`)
   - `days = count(*)`
   - `avg_temp_mean`, `max_temp_max`, `min_temp_min`
   - `total_precip = sum(precipitation)`
   - `wet_days = sum(is_wet_day)`

2. `sw_agg_by_month` (por `month`, acumulando todos los años)
   - `days`, `avg_temp_mean`, `total_precip`, `wet_days`

3. `sw_agg_by_weather` (por `weather`)
   - `days`, `avg_precip`, `avg_wind`, `avg_temp_mean`

> Nota: si quieres, añade un cuarto agregado `year, season` (opcional).

### 5) Persistencia en Postgres
- Escribe **cada dataframe** en su tabla homónima.  
- Modo recomendado: `overwrite` (para que sea re-ejecutable).  
- Nombres de columnas en snake_case.


## Preguntas a responder (en el notebook). 
1) ¿Cuántos días distintos hay en `seattle_weather_raw` y cuál es el **rango de fechas** cubierto?  
2) Según `sw_agg_by_month`, ¿**qué mes** concentra mayor **precipitación total**?  
3) Según `sw_agg_by_year`, ¿en qué **año** hubo mayor **temperatura media** (`temp_mean`)?  
4) ¿Qué categoría de `weather` tiene **más días** y cuál es su **precipitación media**? (usa `sw_agg_by_weather`)  
5) ¿En qué **estación** `season` aparecen más `wet_days` y cuántos son?


---

### 1) Lectura del dataset + configuración previa

In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
#Conexión a la base de datos Postgres
pg_user = "postgres"
pg_password = "mysecretpassword"
pg_host = "localhost"
pg_port = 5432
pg_db = "postgres"

churro = f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
engine = create_engine(churro)

In [3]:
#Descarga del dataset
url = "https://cdn.jsdelivr.net/npm/vega-datasets@3.2.1/data/seattle-weather.csv"
df = pd.read_csv(url)
df.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [4]:
#Ingreso de los datos en raw a la base de datos
df.to_sql("seattle_weather_raw", con=engine, if_exists="replace", index=False)

461

In [5]:
#Comprobación de la carga
df = pd.read_sql("SELECT * FROM seattle_weather_raw;", engine)
df.head(10)

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
5,2012-01-06,2.5,4.4,2.2,2.2,rain
6,2012-01-07,0.0,7.2,2.8,2.3,rain
7,2012-01-08,0.0,10.0,2.8,2.0,sun
8,2012-01-09,4.3,9.4,5.0,3.4,rain
9,2012-01-10,1.0,6.1,0.6,3.4,rain


---
### 2) Limpieza (tareas concretas)

In [6]:
#Eliminar duplicados en date
df_clean = df.drop_duplicates(subset=['date'])

In [7]:
# Cambiamos los valores inválidos
df_clean['precipitation'] = df_clean['precipitation'].where(df_clean['precipitation'] >= 0, 0)
df_clean['wind'] = df_clean['wind'].where(df_clean['wind'] >= 0, 0)  # Elegimos 0 para mantener el mismo tipo de dato y no confundir con un NULL en la columna

# Intercambiamos temp_max y temp_min donde temp_min > temp_max
mask = df_clean['temp_min'] > df_clean['temp_max']
df_clean.loc[mask, ['temp_max', 'temp_min']] = df_clean.loc[mask, ['temp_min', 'temp_max']].values

# Normalización categórica de 'weather'
weather_map = {
    'sun': 'sun',
    'sunny': 'sun',
    'clear': 'sun',
    'rain': 'rain',
    'drizzle': 'drizzle',
    'snow': 'snow',
    'fog': 'fog',
    'drizzle,rain': 'rain',
    'rain,drizzle': 'rain',
    'rain,snow': 'snow',
    'snow,rain': 'snow',
    'fog-rain': 'fog',
    'fog,sun': 'fog',
    'fog,snow': 'fog',
    'fog,drizzle': 'fog',
    'fog,rain': 'fog',
    'fog,snow,rain': 'fog',
    'fog,snow,drizzle': 'fog',
    'fog,snow,rain,drizzle': 'fog'
}

df_clean['weather'] = (
    df_clean['weather']
    .astype(str)
    .str.strip()
    .str.lower()
    .replace(weather_map)
)
df_clean.loc[~df_clean['weather'].isin(['sun', 'rain', 'drizzle', 'snow', 'fog']), 'weather'] = 'other'

df_clean.head(10)

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
5,2012-01-06,2.5,4.4,2.2,2.2,rain
6,2012-01-07,0.0,7.2,2.8,2.3,rain
7,2012-01-08,0.0,10.0,2.8,2.0,sun
8,2012-01-09,4.3,9.4,5.0,3.4,rain
9,2012-01-10,1.0,6.1,0.6,3.4,rain


In [8]:
#Ingreso de los datos limpios a la base de datos
df_clean.to_sql("seattle_weather_clean", con=engine, if_exists="replace", index=False)

461

In [9]:
#Nos aseguramos de que todo está correcto
df_clean = pd.read_sql("SELECT * FROM seattle_weather_clean;", engine)
df_clean.head(10)

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain
5,2012-01-06,2.5,4.4,2.2,2.2,rain
6,2012-01-07,0.0,7.2,2.8,2.3,rain
7,2012-01-08,0.0,10.0,2.8,2.0,sun
8,2012-01-09,4.3,9.4,5.0,3.4,rain
9,2012-01-10,1.0,6.1,0.6,3.4,rain


---
### 3) Nuevas variables

In [10]:
#Extraemos variables desde la fecha
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['dow'] = pd.to_datetime(df['date']).dt.day_name()


In [11]:
#Creamos la variable season a partir del mes
df['season'] = df['month'] % 12 // 3 + 1
season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
df['season'] = df['season'].map(season_map)

In [12]:
df['temp_range'] = df['temp_max'] - df['temp_min']
df['teamp_mean'] = (df['temp_max'] + df['temp_min']) / 2
df['teamp_mean'] = df['teamp_mean'].round(2)
df['is_wet_day'] = df['precipitation'] > 0

df.head(10)

,date,precipitation,temp_max,temp_min,wind,weather,year,month,dow,season,temp_range,teamp_mean,is_wet_day
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle,2012,1,Sunday,Winter,7.8,8.90,False
1,2012-01-02,10.9,10.6,2.8,4.5,rain,2012,1,Monday,Winter,7.8,6.70,True
2,2012-01-03,0.8,11.7,7.2,2.3,rain,2012,1,Tuesday,Winter,4.5,9.45,True
3,2012-01-04,20.3,12.2,5.6,4.7,rain,2012,1,Wednesday,Winter,6.6,8.90,True
4,2012-01-05,1.3,8.9,2.8,6.1,rain,2012,1,Thursday,Winter,6.1,5.85,True
5,2012-01-06,2.5,4.4,2.2,2.2,rain,2012,1,Friday,Winter,2.2,3.30,True
6,2012-01-07,0.0,7.2,2.8,2.3,rain,2012,1,Saturday,Winter,4.4,5.00,False
7,2012-01-08,0.0,10.0,2.8,2.0,sun,2012,1,Sunday,Winter,7.2,6.40,False
8,2012-01-09,4.3,9.4,5.0,3.4,rain,2012,1,Monday,Winter,4.4,7.20,True
9,2012-01-10,1.0,6.1,0.6,3.4,rain,2012,1,Tuesday,Winter,5.5,3.35,True


---
### 4) Tablas agrupadas

In [13]:
#Agrupamos por año y calculamos las estadísticas requeridas
sw_agg_by_year = df.groupby('year').agg(
    days=('date', 'count'),
    avg_temp_mean=('teamp_mean', 'mean'),
    max_temp_max=('temp_max', 'max'),
    min_temp_min=('temp_min', 'min'),
    total_precip=('precipitation', 'sum'),
    wet_days=('is_wet_day', 'sum')
).reset_index()

sw_agg_by_year['avg_temp_mean'] = sw_agg_by_year['avg_temp_mean'].round(2)
sw_agg_by_year['total_precip'] = sw_agg_by_year['total_precip'].round(2)

sw_agg_by_year.head(10)

,year,days,avg_temp_mean,max_temp_max,min_temp_min,total_precip,wet_days
0,2012,366,11.28,34.4,-3.3,1226.0,177
1,2013,365,12.11,33.9,-7.1,828.0,152
2,2014,365,12.83,35.6,-6.0,1232.8,150
3,2015,365,13.13,35.0,-3.8,1139.2,144


In [14]:
#Agrupamos por meses y calculamos las estadísticas requeridas
sw_agg_by_month = df.groupby('month').agg(
    days=('date', 'count'),
    avg_temp_mean=('teamp_mean', 'mean'),
    total_precip=('precipitation', 'sum'),
    wet_days=('is_wet_day', 'sum')
).reset_index()

sw_agg_by_month['avg_temp_mean'] = sw_agg_by_month['avg_temp_mean'].round(2)
sw_agg_by_month['total_precip'] = sw_agg_by_month['total_precip'].round(2)

sw_agg_by_month.head(12)

,month,days,avg_temp_mean,total_precip,wet_days
0,1,124,5.46,466.0,66
1,2,113,6.96,422.0,73
2,3,124,8.62,606.2,73
3,4,120,10.69,375.4,59
4,5,124,14.46,207.5,34
5,6,120,17.32,132.9,37
6,7,124,20.10,48.2,11
7,8,124,20.44,163.7,22
8,9,120,17.14,235.5,35
9,10,124,12.87,503.4,61


In [15]:
#Agrupamos por clima y calculamos las estadísticas requeridas
sw_agg_by_weather = df.groupby('weather').agg(
    days=('date', 'count'),
    avg_precip=('precipitation', 'mean'),
    avg_wind=('wind', 'mean'),
    avg_temp_mean=('teamp_mean', 'mean')
).reset_index()

sw_agg_by_weather['avg_precip'] = sw_agg_by_weather['avg_precip'].round(2)
sw_agg_by_weather['avg_wind'] = sw_agg_by_weather['avg_wind'].round(2)
sw_agg_by_weather['avg_temp_mean'] = sw_agg_by_weather['avg_temp_mean'].round(2)

sw_agg_by_weather.head(10)

,weather,days,avg_precip,avg_wind,avg_temp_mean
0,drizzle,53,0.00,2.37,11.52
1,fog,101,0.00,2.48,12.37
2,rain,641,6.56,3.67,10.52
3,snow,26,8.55,4.41,2.86
4,sun,640,0.00,2.96,14.60


In [16]:
#Agrupamos por estación y calculamos las estadísticas requeridas
sw_agg_by_season = df.groupby('season').agg(
    days=('date', 'count'),
    avg_temp_mean=('teamp_mean', 'mean'),
    max_temp_max=('temp_max', 'max'),
    min_temp_min=('temp_min', 'min'),
    total_precip=('precipitation', 'sum'),
    wet_days=('is_wet_day', 'sum')
).reset_index()

sw_agg_by_season['avg_temp_mean'] = sw_agg_by_season['avg_temp_mean'].round(2)
sw_agg_by_season['total_precip'] = sw_agg_by_season['total_precip'].round(2)

sw_agg_by_season.head(10)

,season,days,avg_temp_mean,max_temp_max,min_temp_min,total_precip,wet_days
0,Fall,364,12.63,33.9,-4.9,1381.4,167
1,Spring,368,11.26,30.6,-1.7,1189.1,166
2,Summer,368,19.31,35.6,6.1,344.8,70
3,Winter,361,6.03,18.9,-7.1,1510.7,220


---
### 5) Persistencia en Postgress

In [17]:
#Cargamos todos los datos procesados a la base de datos
sw_agg_by_year.to_sql("seattle_weather_agg_by_year", con=engine, if_exists="replace", index=False)
sw_agg_by_month.to_sql("seattle_weather_agg_by_month", con=engine, if_exists="replace", index=False)
sw_agg_by_weather.to_sql("seattle_weather_agg_by_weather", con=engine, if_exists="replace", index=False)
sw_agg_by_season.to_sql("seattle_weather_agg_by_season", con=engine, if_exists="replace", index=False)

4

In [18]:
#Comporbamos que todo está correcto
pd.read_sql("SELECT * FROM seattle_weather_agg_by_year;", engine).head(10)

,year,days,avg_temp_mean,max_temp_max,min_temp_min,total_precip,wet_days
0,2012,366,11.28,34.4,-3.3,1226.0,177
1,2013,365,12.11,33.9,-7.1,828.0,152
2,2014,365,12.83,35.6,-6.0,1232.8,150
3,2015,365,13.13,35.0,-3.8,1139.2,144


In [19]:
pd.read_sql("SELECT * FROM seattle_weather_agg_by_weather;", engine).head(10)

,weather,days,avg_precip,avg_wind,avg_temp_mean
0,drizzle,53,0.00,2.37,11.52
1,fog,101,0.00,2.48,12.37
2,rain,641,6.56,3.67,10.52
3,snow,26,8.55,4.41,2.86
4,sun,640,0.00,2.96,14.60


In [20]:
pd.read_sql("SELECT * FROM seattle_weather_agg_by_month;", engine).head(10)

,month,days,avg_temp_mean,total_precip,wet_days
0,1,124,5.46,466.0,66
1,2,113,6.96,422.0,73
2,3,124,8.62,606.2,73
3,4,120,10.69,375.4,59
4,5,124,14.46,207.5,34
5,6,120,17.32,132.9,37
6,7,124,20.10,48.2,11
7,8,124,20.44,163.7,22
8,9,120,17.14,235.5,35
9,10,124,12.87,503.4,61


In [21]:
pd.read_sql("SELECT * FROM seattle_weather_agg_by_season;", engine).head(10)

,season,days,avg_temp_mean,max_temp_max,min_temp_min,total_precip,wet_days
0,Fall,364,12.63,33.9,-4.9,1381.4,167
1,Spring,368,11.26,30.6,-1.7,1189.1,166
2,Summer,368,19.31,35.6,6.1,344.8,70
3,Winter,361,6.03,18.9,-7.1,1510.7,220


---
### 6) Preguntas a responder

1) ¿Cuántos días distintos hay en `seattle_weather_raw` y cuál es el **rango de fechas** cubierto?  

In [22]:
raw = pd.read_sql("SELECT * FROM seattle_weather_raw;", engine)

num_days = raw['date'].nunique()
date_min = raw['date'].min()
date_max = raw['date'].max()

print(f"Días distintos: {num_days}")
print(f"Rango de fechas: {date_min} a {date_max}")

Días distintos: 1461
Rango de fechas: 2012-01-01 a 2015-12-31


2) Según `sw_agg_by_month`, ¿**qué mes** concentra mayor **precipitación total**?  

In [23]:
max_precip_month = sw_agg_by_month.loc[sw_agg_by_month['total_precip'].idxmax()]

print(f"Mes con mayor precipitación total: {max_precip_month['month']} ({max_precip_month['total_precip']} mm)")

Mes con mayor precipitación total: 11.0 (642.5 mm)


3) Según `sw_agg_by_year`, ¿en qué **año** hubo mayor **temperatura media** (`temp_mean`)?  

In [24]:
max_temp_year = sw_agg_by_year.loc[sw_agg_by_year['avg_temp_mean'].idxmax()]

print(f"Año con mayor temperatura media: {int(max_temp_year['year'])} ({max_temp_year['avg_temp_mean']} °C)")

Año con mayor temperatura media: 2015 (13.13 °C)


4) ¿Qué categoría de `weather` tiene **más días** y cuál es su **precipitación media**? (usa `sw_agg_by_weather`)

In [25]:
max_days_weather = sw_agg_by_weather.loc[sw_agg_by_weather['days'].idxmax()]

print(f"Categoría de weather con más días: {max_days_weather['weather']}")
print(f"Precipitación media: {max_days_weather['avg_precip']} mm")

Categoría de weather con más días: rain
Precipitación media: 6.56 mm


5) ¿En qué **estación** `season` aparecen más `wet_days` y cuántos son?

In [26]:
max_wet_season = sw_agg_by_season.loc[sw_agg_by_season['wet_days'].idxmax()]

print(f"Estación con más wet_days: {max_wet_season['season']} ({max_wet_season['wet_days']} días)")

Estación con más wet_days: Winter (220 días)
